In [2]:
import os
import anthropic
from typing import List, Dict, Optional, Any, Tuple, Set, Callable
from dataclasses import dataclass, field
import networkx as nx
import json
import time
import uuid
from dotenv import load_dotenv
import sys
load_dotenv()

# Añadir el directorio raíz al path para importar los módulos
sys.path.append(os.path.dirname(os.path.abspath("../")))


In [3]:
from incubator.wf.wf import WorkflowEngine, Node
from incubator.wf.node import Node

from incubator.llm.antropic import AnthropicClient
from incubator.agents.agent import Agent


In [4]:
# Example: Implementation for a trading strategy workflow
def create_trading_strategy_workflow(api_key: Optional[str] = None) -> WorkflowEngine:
    """
    Create a workflow for generating trading strategies
    
    Args:
        api_key: Anthropic API key (optional)
        
    Returns:
        Configured WorkflowEngine
    """
    # Create the LLM client
    llm_client = AnthropicClient(api_key)
    
    # Create the workflow engine
    workflow = WorkflowEngine()
    
    # Define agents
    explorer = Agent(
        name="Explorer",
        description="Generates original quantitative strategy ideas",
        llm_client=llm_client,
        system_prompt="""
        You are the Explorer Agent in a quantitative strategy incubation system.
        Your specialty is generating innovative algorithmic strategies focused on the S&P 500.
        
        IMPORTANT: Generate ONE quantitative strategy idea per iteration, not multiple.
        
        Your function is to:
        
        1. Generate ONE innovative and detailed quantitative strategy idea for trading the S&P 500
        2. Explain the mathematical/statistical foundations and how the strategy works
        3. Detail specific signals, timeframes, factors, and metrics to use
        4. Highlight potential advantages of the strategy (alpha, Sharpe, drawdown, etc.)
        5. Respond to questions or suggestions from the Curator
        
        Consider aspects such as:
        - Market factors (momentum, value, volatility, etc.)
        - Statistical analyses (cointegration, regression, clustering)
        - Innovative technical or fundamental indicators
        - Execution optimization and transaction cost management
        - Risk management and strategy diversification
        - Avoid hardcoded values
        - Consider self-adaptation and statistical or ML predictive models
        - Avoid thresholds or hardcoded values, use optimizations, inferences, or self-adaptive strategies
        
        Technical limitations:
        - The strategy should be implementable in Python using the yfinance library for data
        - Do not provide code, only the logic and detailed methodology
        - The idea will move to the next development level where it will be implemented
        
        Be specific, technical, and detailed. Think of implementable and backtestable strategies.
        """,
        config={
            "temperature": 0.8,
            "use_thinking": False,
            "max_tokens": 15000
        }
    )
    
    curator = Agent(
        name="Curator",
        description="Evaluates quantitative strategies and selects the most promising one",
        llm_client=llm_client,
        system_prompt="""
        You are the Curator Agent in a quantitative strategy incubation system.
        You have extensive experience evaluating algorithmic strategies focused on the S&P 500.
        Your function is to evaluate the strategy proposed by the Explorer and suggest specific improvements. You must:
        
        1. Critically analyze the strategy considering:
           - Expected Sharpe ratio and statistical robustness
           - Ability to generate true alpha (not disguised beta)
           - Scalability and capacity (how much capital it can handle)
           - Implementation and transaction costs
           - Exposure to known risk factors
           - Risk of overoptimization or data snooping
           - Feasibility of implementation with yfinance in Python
           - Ease of implementation and evaluation
        
        2. Propose specific technical improvements such as:
           - Backtest and walk forward analysis
           - Avoiding look-ahead bias
           - Parameter or signal refinement
           - Risk management improvements
           - Execution optimization
           - Complements with other factors or signals
           - Avoiding thresholds or hardcoded values, use optimizations, inferences, or self-adaptive strategies
        
        3. Formulate specific technical questions to clarify ambiguous aspects
        
        Technical limitations:
        - The strategy should be implementable in Python using the yfinance library for data
        - Do not provide code, only conceptual improvements and recommendations
        - The idea will move to the next development level where it will be implemented
        
        Maintain a rigorous and skeptical approach, as an experienced risk manager would.

        YOUR TASK IS TO ITERATE THE IDEA AND DESCRIBE THE FINAL IDEA BUT AVOID SHOWING CODE AS MUCH AS POSSIBLE.
        DEVELOPING THE CODE IS THE TASK OF ANOTHER SPECIALIZED DEVELOPMENT AGENT.
        
        IMPORTANT: If this is the final iteration (you will be informed in the message), you must finish with a
        detailed and technically sound version of the strategy, ready to be implemented in Python with yfinance.
        In that case, include:
           - Exact entry/exit logic
           - Specific parameters
           - Risk management
           - Performance expectations
           - Metrics
           - Backtest and walk-forward, look-ahead bias, etc.
           - Technical implementation considerations
        
        When providing the final version, start your response with "FINAL IDEA:".
        """,
        config={
            "temperature": 0.5,
            "use_thinking": True,
            "thinking_budget": 16000,
            "max_tokens": 32000
        }
    )
    
    developer = Agent(
        name="Developer",
        description="Implements the strategy in Python code using yfinance",
        llm_client=llm_client,
        system_prompt="""
        You are the Developer Agent in a quantitative strategy implementation system.
        Your specialty is converting strategy ideas into functional Python code using the yfinance library.
        
        You must implement the provided strategy as a complete and functional Python program. Your code should:
        
        1. Use the yfinance library to obtain S&P 500 data
        2. Implement the exact logic described in the strategy
        3. Generate and save performance metrics, charts, and analysis
        4. Avoid thresholds and hardcoded parameters, use optimization or self-adaptive inference
        5. Handle errors and validate data properly
        6. Be well documented with clear comments
        7. Implement all described methods including backtest and walk-forward
        8. Avoid look-ahead bias and biases when using future data that you wouldn't have in a real implementation
        9. Be concise in the code at this stage
        
        NOTE TO THE DEVELOPER
        yfinance has auto_adjust=True by default in the installed version
        recent example of yfinance version installed yf.download(stock, start=start_date, end=end_date)['Close'] # auto_adjust=True by default
        
        # Create directories for results
        os.makedirs('./artifacts/results', exist_ok=True)
        os.makedirs('./artifacts/results/figures', exist_ok=True)
        os.makedirs('./artifacts/results/data', exist_ok=True)
        
        # Configure logging
        logging.basicConfig(
            filename='./artifacts/errors.txt',
            level=logging.ERROR,
            format='[%(asctime)s] %(levelname)s: %(message)s',
            datefmt='%Y-%m-%d %H:%M:%S'
        )

        **Pay attention to typical pandas series errors with scalars and NaN**
        
        IMPORTANT - OUTPUT FORMAT:
        1. Present your code within <python>CODE</python> tags
        2. All results, charts, and output files such as metrics in csv or txt should be saved in the './artifacts/results/' folder
        3. Capture errors and save them in the './artifacts/errors.txt' file
        
        Your code must ensure that:
        - All necessary folders are created if they don't exist
        - Performance metrics (Sharpe, drawdown, etc.) are saved in CSV files
        - Visualizations (performance charts, signals, etc.) are generated
        - Errors are properly handled and logged in a log file within './artifacts/errors.txt'
        
        **provide a full error traceback into ./artifacts/error.txt for future improvements**
        
        Provide a complete Python program, ready to run, that optimally implements the strategy.
        """,
        config={
            "temperature": 0.2,
            "use_thinking": False,
            "max_tokens": 25000
        }
    )
    
    # Create nodes
    explorer_node = Node(id="explorer", agent=explorer)
    curator_node = Node(id="curator", agent=curator)
    developer_node = Node(id="developer", agent=developer)
    
    # Add nodes to the workflow
    workflow.add_node(explorer_node, is_input=True, is_output=False)
    workflow.add_node(curator_node, is_input=False, is_output=False)
    workflow.add_node(developer_node, is_input=False, is_output=True)
    
    # Add edges to define the communication flow
    workflow.add_edge("explorer", "curator")
    workflow.add_edge("curator", "explorer")  # Feedback loop
    workflow.add_edge("curator", "developer")  # Final output goes to developer
    
    return workflow

# Example: Create and use the trading strategy workflow from before
strategy_workflow = create_trading_strategy_workflow()

strategy_inputs = {
    "explorer": """
    I want to generate a dataset of unique and genuine features. The objective is to later train a ranking model,
    but I want to generate a list of unique features for each stock in the S&P 500. Please design the features 
    and unique calculations.
    """
}

strategy_results = strategy_workflow.execute(strategy_inputs)

# Save configurations
workflow.save("content_analysis_workflow.json")
strategy_workflow.save("trading_strategy_workflow.json")    

Error: Workflow contains cycles


ValueError: Invalid workflow graph